# GRIB2-CMA GEPS
This notebook provides an example on how to decode GRIB2 data using the pygrib library. In this example, we will read the Global Ensemble Prediction System of China Meteorological Administration (CMA-GEPS) and display one of the parameters on maps.<br>
<br>
The following references were used to create this notebook:<br>
- Sample data from <https://wis2node.wis.cma.cn/oapi/collections/discovery-metadata/items/urn:wmo:md:cn-cma:data.core.weather.prediction.forecast.medium-range.probabilistic.global>
- Python code to load GRIB data, see <https://pypi.org/project/pygrib/><br>

## About the data

Sample data is included in the base-image of [demo-decode-eccodes-jupyter](https://github.com/wmo-im/demo-decode-eccodes-jupyter).<br>
<br>
__Data Description__<br>
The Global Ensemble Prediction System of China Meteorological Administration (CMA-GEPS) carries out physics calculations to arrive at probabilistic predictions of atmospheric elements from the current day out to 15 days. The CMA-GEPS produces different forecasts to estimate the forecast uncertainties due to the nonlinear (chaotic) nature of the atmosphere. The probabilistic predictions are based on an ensemble of 30 perturbed members that differ in their initial conditions with Singular Vectors (SVs), their model uncertainties which are randomly perturbed by Stochastic Physics Perturbation Tendency (SPPT) method, and the Stochastic Kinetic Energy Backscatter (SKEB). A control member that is not perturbed is also available. <br>
Weather elements include temperature, precipitation, wind speed, gust and others. <br>
This product contains raw numerical results of these members. <br>
Geographical coverage is global. Data is available on a global latitude-longitude uniform grid with 0.5 degree horizontal resolution. <br>
Predictions are performed twice a day and initial conditions are at 00 UTC and 12 UTC.<br>

## Dependencies
This notebook uses several Python modules for reading GRIB data, numerical computing, visualization, and cartographic mapping:  

The following modules are imported:  

- **pygrib**: Python interface for reading and writing GRIB (Gridded Binary) files, enabling extraction of meteorological data (e.g., loading forecast fields like temperature or wind).  
- **numpy**: Numerical Python library for efficient array operations and mathematical computations (e.g., `np.array` for handling gridded data).  
- **matplotlib**: Plotting library for creating static, interactive, or animated visualizations (e.g., `plt` for rendering maps or graphs).  
- **cartopy**: Cartographic library for geospatial data visualization, offering map projections and geographic features (e.g., `ccrs` for coordinate reference systems, `cfeature` for adding coastlines or borders).  

This setup is typically used for analyzing and visualizing meteorological or climate data stored in GRIB format, with **pygrib** handling data ingestion and the remaining modules supporting processing and mapping tasks.

## About **pygrib**  
**Python library for reading GRIB files (editions 1 and 2)**, enabling access to meteorological and climate data stored in GRIB format. Requires the **ECCODES C library** as a dependency.

**If you are running the Jupyter environment provided by 'demo-decode-eccodes-jupyter', pygrib is already installed.** If you are running a local Jupyter environment, please follow the instructions below to install pygrib.

## Installing pygrib on a local Jupyter environment

### **Recommended Installation (Conda)**  
For Anaconda/Miniconda users, the simplest method is:  
```bash
conda install -c conda-forge pygrib
```  
This automatically resolves dependencies (e.g., ECCODES, numpy, pyproj).  

### **Verification**  
Test installation with:  
```python
import pygrib
print(pygrib.__version__)  # Should output the installed version (e.g., `2.1.6`)
```  

## Getting started

The first code block import pygrib and displays the versions of pygrib and eccodes to verify the installation. It also sets the path to the GRIB2 file to be read. By default, it uses a sample GRIB2 file included in the Docker image. If you are running this notebook on your local machine, please change the path to point to your downloaded GRIB2 file.

The first code block also defines the dictionary specific to the CMA-GEPS that specific per message that describes the type of probability forecast. This dictionary is used later in the notebook to interpret the GRIB messages.

Execute each block of code by selecting it and pressing ``Shift+Enter`` or clicking the ``Run`` button in the toolbar above.

In [ ]:
# first import the necessary libraries and check versions
import pygrib
print("pygrib version:")
print(pygrib.__version__) 
print("eccodes version:")
print(pygrib.eccodes_version)

# Path to your GRIB2 file, default using sample data included in the Docker image
grib_file = "/root/sample-data/Z_NAFP_C_BABJ_20250818000000_P_CMA-GEPS-GLB-024.grib2"

# CMA GEPS=GRAPES data specific mapping per message-type
msg_description = {
    1: "Probability of 24-hour cumulative precipitation exceeding 1mm",
    2: "Probability of 24-hour cumulative precipitation exceeding 5mm",
    3: "Probability of 24-hour cumulative precipitation exceeding 10mm",
    4: "Probability of 24-hour cumulative precipitation exceeding 25mm",
    5: "Probability of 24-hour cumulative precipitation exceeding 50mm",
    6: "Probability of 24-hour cumulative precipitation exceeding 100mm",
    7: "Probability of 850hPa temperature anomaly below -2 standard deviations",
    8: "Probability of 850hPa temperature anomaly below -1.5 standard deviations",
    9: "Probability of 850hPa temperature anomaly below -1 standard deviation",
    10: "Probability of 850hPa temperature anomaly exceeding 1 standard deviation",
    11: "Probability of 850 hPa temperature anomaly exceeding 1.5 standard deviations",
    12: "Probability of 850 hPa temperature anomaly exceeding 2 standard deviations",
    13: "Probability of 10-metre wind speed exceeding 10 m/s",
    14: "Probability of 10-metre wind speed exceeding 15 m/s",
    15: "Probability of 10-metre wind speed exceeding 25 m/s",
    16: "Probability of 10-metre gusts exceeding 10 m/s",
    17: "Probability of 10-metre gusts exceeding 15 m/s",
    18: "Probability of 10-metre gusts exceeding 25 m/s",
    19: "Ensemble mean of 500 hPa geopotential height",
    20: "Dispersion of 500 hPa geopotential height",
    21: "Sea-level pressure ensemble mean",
    22: "Sea-level pressure dispersion",
    23: "850 hPa wind speed ensemble mean",
    24: "850 hPa wind speed dispersion",
    25: "250 hPa wind speed ensemble mean",
    26: "250 hPa wind speed dispersion"
}

# Review contents of the GRIB2 file

The next code block opens the GRIB2 file and lists all messages (data fields) contained in the file and prints some metadata for each message.

In [ ]:
# the following code demonstrates how to display information about the data contained in the GRIB2 file
grbs = pygrib.open(grib_file)
header = f"{'Msg':<3} {'Parameter':<25} {'Units':<8} {'LevelType':<18} {'Level':<6} {'Fcst':<5} {'Step':<6} {'Description':<50}"
print(header)
print("-" * len(header))
for grb in grbs:
    print(f"{grb.messagenumber:<3} "
          f"{grb.parameterName:<25}"
          f"{grb.parameterUnits:<8}"
          f"{grb.typeOfLevel:<18} "
          f"{grb.level:<6} "
          f"{grb.forecastTime:<5} "
          f"{grb.stepRange:<6} "
          f"{msg_description.get(grb.messagenumber)}")

# Visualization example

The following code block shows an example how to visualize one of the probabilistic predictions of Temperature Anomaly on a map using Cartopy and Matplotlib.

It sets up a map projection, adds geographic features, and plots the data with appropriate color mapping and legends.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from matplotlib.colorbar import ColorbarBase
import pygrib

def show_map_visualization(grib_file, my_message):
    # Open GRIB file
    grbs = pygrib.open(grib_file)

    my_grb = None
    for grb in grbs:
        if my_message == grb.messagenumber:
            my_grb = grb
            break

    if my_grb is None:
        grbs.close()
        raise ValueError(f"No message found with messagenumber={my_message}")

    # Extract data
    data, lats, lons = my_grb.data()
    data_min, data_max = np.nanmin(data), np.nanmax(data)

    # Create figure
    fig, ax = plt.subplots(
        figsize=(10, 6),
        subplot_kw={"projection": ccrs.PlateCarree()}
    )

    # Ensure global display
    ax.set_global()

    # Add map features
    ax.add_feature(cfeature.COASTLINE, linewidth=0.5)
    ax.add_feature(cfeature.BORDERS, linestyle=":", linewidth=0.5)
    ax.add_feature(cfeature.LAND, edgecolor="black", facecolor="lightgray")

    # Plot data
    cmap = "YlOrRd"
    mesh = ax.pcolormesh(
        lons, lats, data,
        cmap=cmap,
        vmin=data_min, vmax=data_max,
        transform=ccrs.PlateCarree()
    )

    # Create horizontal colorbar
    pos = ax.get_position()
    cax = fig.add_axes([pos.x0, pos.y0 - 0.08, pos.width, 0.02])  # adjust height
    norm = plt.Normalize(vmin=data_min, vmax=data_max)
    ColorbarBase(cax, cmap=cmap, norm=norm, orientation='horizontal', extend='both')
    cax.set_title(f"{my_grb.parameterName} ({my_grb.units})", fontsize=10)

    # Display min/max values
    ax.text(
        0.5, -0.25,
        f"Min: {data_min:.2f}\nMax: {data_max:.2f}",
        transform=ax.transAxes,
        ha="center", va="center",
        fontsize=10,
        bbox=dict(facecolor="white", alpha=0.8, edgecolor="gray", boxstyle="round")
    )

    # Set title
    ax.set_title(f"{my_grb.parameterName}\nValid: {my_grb.validDate}", fontsize=12)

    # Main figure title
    plt.suptitle(f"{msg_description.get(my_grb.messagenumber)}", fontsize=16, y=0.95)

    # Display plot
    plt.show()

    grbs.close()

show_map_visualization(grib_file, 8)